## Lab 6: Intrinsic Dimension and Density Estimation
You can use external libraries for linear algebra operations but you are expected to write your own algorithms.

# Exercise 1
Using the ```dry_beans_dataset``` as we did in previous laboratories (ie. follow the same proprocessing steps but **do not** perform the train-test split), program your own implementation of the two-NN estimate for the Intrinsic Dimension. 

Is the result comparible with what you would expect from an analysis of PCA's spectrum?

In [ ]:
### solutions will be available next week

# Exercise 2
Using the following code, create a one-dimensional dataset of size $N=100$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, t #normal distribution, t distribution

np.random.seed(44)

In [ ]:
N = 100

X = np.concatenate(
    (np.random.standard_t(1, int(0.04*N))-3.5,np.random.normal(5, 1, int(0.48 * N)), np.random.normal(7.5, 1, int(0.48 * N)))
)[:, np.newaxis]

In [ ]:
X.shape

In [ ]:
X_plot = np.linspace(-12,12, 1000)[:, np.newaxis]
true_dens = 0.04* t(df=1,loc=-3.5).pdf(X_plot[:, 0]) + 0.48* norm(5, 1).pdf(X_plot[:, 0]) + 0.48*norm(7.5,1).pdf(X_plot[:,0])

fig = plt.figure(figsize=(10,8))
plt.fill(X_plot[:, 0], true_dens, fc="black", alpha=0.2)
plt.show()

Compute the density estimation with your implementations of:
- Histogram Density Estimation (Freedman-Diaconis rule)

In [ ]:
### solutions will be available next week

- Kernel Density Estimation (KDE) - Gaussian kernel (Silverman's rule)

In [ ]:
### solutions will be available next week